<a href="https://colab.research.google.com/github/Haruka0522/Small-Programs/blob/master/practice_gan/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%tensorflow_version 1.x

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Input, Multiply, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model, Sequential
from keras.optimizers import Adam

モデルを定義

In [ ]:
#入力
img_row = 28
img_col = 28
channels = 1

img_shape = (img_row,img_col,channels)

z_dim = 100
num_classes = 10  #データセットのクラス数

In [6]:
#Gモデル
def build_generator(z_dim):
  model = Sequential()
  model.add(Dense(256,input_shape=(z_dim,),activation="relu"))
  model.add(Dense(512, activation="relu"))
  model.add(Dense(1024, activation="relu"))
  model.add(Dense(np.prod(img_shape),activation="tanh"))

  model.add(Reshape(img_shape))

  model.summary()
  return model

In [8]:
#Dモデル
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(256,activation="relu"))
    model.add(Dense(1,activation="relu"))
    model.summary()
    return model

In [10]:
#DとGの合併
def build_combined1():
    discriminator.trainable = False
    model = Sequential([generator,discriminator])
    return model

In [11]:
optimizer = Adam(0.0002,0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
#Dモデル構築
discriminator = build_discriminator(img_shape)
discriminator.compile(loss="binary_crossentropy",
                      optimizer=optimizer,
                      metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Gモデル構築
generator = build_generator(z_dim)
combined = build_combined1()
combined.compile(loss="binary_crossentropy",
                 optimizer=optimizer)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_7 (Dense)              (None, 784)               803600    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
Total params: 1,486,352
Trainable params: 1,486,352
Non-trainable params: 0
_________________________________________________________________
